In [1]:
import pandas as pd
import numpy as np
from docplex.mp.model import Model

In [23]:
# Sudoku parameters
N_DIGITS = 9
N_ROWS = 9
N_COLS = 9
GRID_SIZE = N_ROWS * N_COLS

In [37]:
# Hard
grid = np.array(
    [
        [0, 0, 0, 0, 5, 1, 0, 0, 0],
        [0, 0, 2, 3, 0, 0, 1, 0, 9],
        [0, 0, 1, 9, 8, 2, 3, 0, 0],
        [0, 8, 4, 0, 3, 7, 0, 0, 0],
        [0, 6, 0, 0, 4, 0, 0, 7, 0],
        [9, 0, 0, 0, 0, 0, 4, 0, 0],
        [2, 0, 0, 5, 0, 0, 6, 0, 0],
        [0, 4, 0, 0, 2, 0, 0, 9, 0],
        [0, 0, 0, 1, 0, 0, 0, 4, 0]
    ]
)
# Specialist
grid = np.array(
    [
        [0, 0, 0, 0, 9, 0, 0, 2, 0],
        [4, 0, 2, 5, 0, 0, 0, 6, 0],
        [0, 5, 3, 0, 7, 0, 0, 4, 0],
        [0, 7, 8, 0, 0, 1, 0, 0, 0],
        [9, 0, 0, 0, 5, 0, 0, 0, 0],
        [0, 4, 0, 6, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 7, 0, 0, 2],
        [5, 0, 0, 0, 4, 0, 7, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 6]
    ]
)
grid.shape

# Model:

Maximize $0$

Subject to:
- $\sum_{k=1}^{9}{x(i, j, k)} = 1 \quad \forall i, j = 1 \dots 9$ (1)
- $\sum_{j=1}^{9}{x(i, j, k)} = 1 \quad \forall i, k = 1 \dots 9$ (2)
- $\sum_{i=1}^{9}{x(i, j, k)} = 1 \quad \forall j, k = 1 \dots 9$ (3)
- $\sum_{i=1}^{3}\sum_{j=1}^{3}x(i, j, k) = 1 \quad \forall \quad 1 \le i \le 3 \quad , \forall \quad 1 \le k \le 9$ (4)

or:

- $\sum_{i=1}^{3}\sum_{j=1}^{3}x(i + U, j + V, k) = 1 \quad where \quad U, V \in [0, 3, 6]$ (4)

In [39]:
# Model
mdl = Model(name='Sudoku')

# Decision variables
keys = []
for i in range(N_ROWS):
    for j in range(N_COLS):
        for k in range(N_DIGITS):
            keys.append(f'x_({i},{j},{k})')
            
x = np.array(mdl.binary_var_list(keys)).reshape((9, 9, 9))

# Constraints:

# Preset grid number
for i in range(N_ROWS):
    for j in range(N_COLS):
        k = grid[i, j]
        if k != 0:
            mdl.add_constraint(x[i, j, (k - 1)] == 1)


# (1) Only one digit must be selected - Depth must sum 1
for i in range(N_ROWS):
    for j in range(N_COLS):
        mdl.add_constraint(mdl.sum([x[i, j, k] for k in range(N_DIGITS)]) == 1)

# (3) Can't repeat same digit in columns - Columns must sum 1 in all depths
for j in range(N_COLS):
    for k in range(N_DIGITS):
        mdl.add_constraint(mdl.sum([x[i, j, k] for i in range(N_ROWS)]) == 1)
        
# (2) Can't repeat same digit in rows - Rows must sum 1 in all depths
for i in range(N_ROWS):
    for k in range(N_DIGITS):
        mdl.add_constraint(mdl.sum([x[i, j, k] for j in range(N_COLS)]) == 1)
        
# (4) Can't repeat same digit in 3x3 inner grids - Each 3x3 inner grid must sum 1 in all depths
for i in range(0, N_ROWS, 3):
    for j in range(0, N_COLS, 3):
        for k in range(N_DIGITS):
            mdl.add_constraint(mdl.sum([x[row, col, k] for row in range(i, i+3) for col in range(j, j+3)]) == 1)
            
# Objective Function
mdl.minimize(0)

# Solve model
mdl.solve()

# Print solution
x_sol = []
for i in range(N_ROWS):
    for j in range(N_COLS):
        for k in range(N_DIGITS):
            if x[i, j, k].solution_value == 1:
                x_sol.append(k+1)
x_sol = np.array(x_sol).reshape((9, 9))
print(x_sol)

[[6 1 7 8 9 4 5 2 3]
 [4 9 2 5 1 3 8 6 7]
 [8 5 3 2 7 6 9 4 1]
 [2 7 8 4 3 1 6 9 5]
 [9 3 6 7 5 8 2 1 4]
 [1 4 5 6 2 9 3 7 8]
 [3 8 9 1 6 7 4 5 2]
 [5 6 1 3 4 2 7 8 9]
 [7 2 4 9 8 5 1 3 6]]
